In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [2]:
source_dirs = ["/shared/ds/apps/DSpackages/segmentation_models.pytorch"]
working_dir = "/shared/ds/data/compvis/maize/PLTQR_explainable/"
model_dir = os.path.join(working_dir, "models")

In [3]:
import torch
import sys
from torchinfo import summary

sys.path.extend(source_dirs)
import segmentation_models_pytorch as smp

## Load model and save

In [208]:
model_resnet18 = smp.Unet(
    encoder_name="resnet18",  # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
    encoder_weights="imagenet",  # use `imagenet` pre-trained weights for encoder initialization
    in_channels=1,  # model input channels (1 for gray-scale images, 3 for RGB, etc.)
    classes=3,  # model output channels (number of classes in your dataset)

)

In [209]:
modules = [m  for m in model_resnet18.modules()]
print(len(modules))

print(modules[1])


140
ResNetEncoder(
  (conv1): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inpla

In [51]:
print(len(modules))

140


In [75]:
with torch.no_grad():
    out = model_resnet18.encoder.conv1(torch.randn(1, 1, 576, 576))

In [76]:
print(out.shape)

torch.Size([1, 64, 288, 288])


In [ ]:
model_resnet18.encoder.conv1(torch.randn(1, 1, 576, 576))

In [87]:
import torch.nn as nn

class myNet(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv = nn.Conv2d(3,10,2, stride = 2)
    self.relu = nn.ReLU()
    self.flatten = lambda x: x.view(-1)
    self.fc1 = nn.Linear(160,5)
  def forward(self, x):
    x = self.relu(self.conv(x))
    return self.fc1(self.flatten(x))

net = myNet()

def hook_fn(m, i, o):
  print(m)
  print("------------Input Grad------------")
  for grad in i:
    try:
      print(grad.shape)
    except AttributeError:
      print ("None found for Gradient")

  print("------------Output Grad------------")
  for grad in o:
    try:
      print(grad.shape)
      print(grad)
    except AttributeError:
      print ("None found for Gradient")
  print("\n")
net.conv.register_backward_hook(hook_fn)
net.fc1.register_backward_hook(hook_fn)
inp = torch.randn(1,3,8,8)
out = net(inp)

(1 - out.mean()).backward()

Linear(in_features=160, out_features=5, bias=True)
------------Input Grad------------
torch.Size([5])
torch.Size([5])
------------Output Grad------------
torch.Size([5])
tensor([-0.2000, -0.2000, -0.2000, -0.2000, -0.2000])


Conv2d(3, 10, kernel_size=(2, 2), stride=(2, 2))
------------Input Grad------------
None found for Gradient
torch.Size([10, 3, 2, 2])
torch.Size([10])
------------Output Grad------------
torch.Size([1, 10, 4, 4])
tensor([[[[ 0.0000,  0.0349,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000, -0.0153],
          [ 0.0000, -0.0024,  0.0041,  0.0000],
          [ 0.0084, -0.0032,  0.0000,  0.0288]],

         [[ 0.0000,  0.0000,  0.0077, -0.0091],
          [ 0.0000,  0.0000,  0.0032,  0.0000],
          [-0.0081,  0.0000,  0.0000,  0.0162],
          [ 0.0000,  0.0000,  0.0000,  0.0000]],

         [[-0.0230,  0.0222,  0.0000, -0.0021],
          [ 0.0166,  0.0170, -0.0316,  0.0000],
          [-0.0171,  0.0000,  0.0391,  0.0116],
          [ 0.0000,  0.0000, 

In [108]:
aaaa = None
names, modules= [], []
for ii, (name, module) in enumerate(model_resnet18.named_modules()):
    #print(ii,aa)
    names.append(name)
    modules.append(module)

    if ii == 104:
        aaaa = module


In [113]:
names

['',
 'encoder',
 'encoder.conv1',
 'encoder.bn1',
 'encoder.relu',
 'encoder.maxpool',
 'encoder.layer1',
 'encoder.layer1.0',
 'encoder.layer1.0.conv1',
 'encoder.layer1.0.bn1',
 'encoder.layer1.0.relu',
 'encoder.layer1.0.conv2',
 'encoder.layer1.0.bn2',
 'encoder.layer1.1',
 'encoder.layer1.1.conv1',
 'encoder.layer1.1.bn1',
 'encoder.layer1.1.relu',
 'encoder.layer1.1.conv2',
 'encoder.layer1.1.bn2',
 'encoder.layer2',
 'encoder.layer2.0',
 'encoder.layer2.0.conv1',
 'encoder.layer2.0.bn1',
 'encoder.layer2.0.relu',
 'encoder.layer2.0.conv2',
 'encoder.layer2.0.bn2',
 'encoder.layer2.0.downsample',
 'encoder.layer2.0.downsample.0',
 'encoder.layer2.0.downsample.1',
 'encoder.layer2.1',
 'encoder.layer2.1.conv1',
 'encoder.layer2.1.bn1',
 'encoder.layer2.1.relu',
 'encoder.layer2.1.conv2',
 'encoder.layer2.1.bn2',
 'encoder.layer3',
 'encoder.layer3.0',
 'encoder.layer3.0.conv1',
 'encoder.layer3.0.bn1',
 'encoder.layer3.0.relu',
 'encoder.layer3.0.conv2',
 'encoder.layer3.0.bn2',


In [114]:
dir(modules[0])

['__call__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_apply',
 '_backward_hooks',
 '_buffers',
 '_forward_hooks',
 '_forward_pre_hooks',
 '_get_name',
 '_load_from_state_dict',
 '_load_state_dict_pre_hooks',
 '_modules',
 '_named_members',
 '_parameters',
 '_register_load_state_dict_pre_hook',
 '_register_state_dict_hook',
 '_replicate_for_data_parallel',
 '_save_to_state_dict',
 '_slow_forward',
 '_state_dict_hooks',
 '_version',
 'add_module',
 'apply',
 'bfloat16',
 'buffers',
 'check_input_shape',
 'children',
 'classification_head',
 'cpu',
 'cuda',
 'decoder',
 'double',
 'dump_patches',
 'encoder',
 'eval',
 'extra_repr'

In [122]:
print(modules[1].named_children)

<bound method Module.named_children of Unet(
  (encoder): ResNetEncoder(
    (conv1): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=

In [112]:
len(set(names))

140

In [221]:
model_resnet18 = smp.Unet(
    encoder_name="resnet18",  # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
    encoder_weights="imagenet",  # use `imagenet` pre-trained weights for encoder initialization
    in_channels=1,  # model input channels (1 for gray-scale images, 3 for RGB, etc.)
    classes=3,  # model output channels (number of classes in your dataset)

)
net = model_resnet18
visualisation = []

def hook_fn(m, i, o):
  visualisation.append((m.nv_name,m, o))
  #visualisation[m] = o

def get_all_layers(net):
  for name, layer in net.named_modules():
    #print(name)
    #print(type(layer))
    #If it is a sequential, don't register a hook on it
    # but recursively register hook on all it's module children
    if 1==1: # if not isinstance(layer, nn.Sequential):
    #  get_all_layers(layer)
    #else:
      # it's a non sequential. Register a hook
        print(name)
        layer.nv_name=name
        layer.register_forward_hook(hook_fn)


get_all_layers(net)


out = net(torch.randn(1, 1, 576, 576))

# Just to check whether we got all layers
#visualisation.keys()      #output includes sequential layers


encoder
encoder.conv1
encoder.bn1
encoder.relu
encoder.maxpool
encoder.layer1
encoder.layer1.0
encoder.layer1.0.conv1
encoder.layer1.0.bn1
encoder.layer1.0.relu
encoder.layer1.0.conv2
encoder.layer1.0.bn2
encoder.layer1.1
encoder.layer1.1.conv1
encoder.layer1.1.bn1
encoder.layer1.1.relu
encoder.layer1.1.conv2
encoder.layer1.1.bn2
encoder.layer2
encoder.layer2.0
encoder.layer2.0.conv1
encoder.layer2.0.bn1
encoder.layer2.0.relu
encoder.layer2.0.conv2
encoder.layer2.0.bn2
encoder.layer2.0.downsample
encoder.layer2.0.downsample.0
encoder.layer2.0.downsample.1
encoder.layer2.1
encoder.layer2.1.conv1
encoder.layer2.1.bn1
encoder.layer2.1.relu
encoder.layer2.1.conv2
encoder.layer2.1.bn2
encoder.layer3
encoder.layer3.0
encoder.layer3.0.conv1
encoder.layer3.0.bn1
encoder.layer3.0.relu
encoder.layer3.0.conv2
encoder.layer3.0.bn2
encoder.layer3.0.downsample
encoder.layer3.0.downsample.0
encoder.layer3.0.downsample.1
encoder.layer3.1
encoder.layer3.1.conv1
encoder.layer3.1.bn1
encoder.layer3.1.re

In [275]:
class VerboseExecution(nn.Module):
    def __init__(self, model: nn.Module):
        super().__init__()
        self.model = model

        # Register a hook for each layer
        for name, layer in self.model.named_modules():
            layer.__name__ = name
            layer.register_forward_hook(
                lambda layer, _, output: print(f"{layer.__name__}: {output.shape} " if hasattr(output,'shape') else f"{layer.__name__}: NO SHAPE" )
            )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self.model(x)

In [276]:
model_resnet18 = smp.Unet(
    encoder_name="resnet18",  # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
    encoder_weights="imagenet",  # use `imagenet` pre-trained weights for encoder initialization
    in_channels=1,  # model input channels (1 for gray-scale images, 3 for RGB, etc.)
    classes=3,  # model output channels (number of classes in your dataset)
)
verbose_model_resnet18 = VerboseExecution(model_resnet18)
dummy_input = torch.randn(1, 1, 576, 576)

_ = verbose_model_resnet18(dummy_input)


encoder.conv1: torch.Size([1, 64, 288, 288]) 
encoder.bn1: torch.Size([1, 64, 288, 288]) 
encoder.relu: torch.Size([1, 64, 288, 288]) 
encoder.maxpool: torch.Size([1, 64, 144, 144]) 
encoder.layer1.0.conv1: torch.Size([1, 64, 144, 144]) 
encoder.layer1.0.bn1: torch.Size([1, 64, 144, 144]) 
encoder.layer1.0.relu: torch.Size([1, 64, 144, 144]) 
encoder.layer1.0.conv2: torch.Size([1, 64, 144, 144]) 
encoder.layer1.0.bn2: torch.Size([1, 64, 144, 144]) 
encoder.layer1.0.relu: torch.Size([1, 64, 144, 144]) 
encoder.layer1.0: torch.Size([1, 64, 144, 144]) 
encoder.layer1.1.conv1: torch.Size([1, 64, 144, 144]) 
encoder.layer1.1.bn1: torch.Size([1, 64, 144, 144]) 
encoder.layer1.1.relu: torch.Size([1, 64, 144, 144]) 
encoder.layer1.1.conv2: torch.Size([1, 64, 144, 144]) 
encoder.layer1.1.bn2: torch.Size([1, 64, 144, 144]) 
encoder.layer1.1.relu: torch.Size([1, 64, 144, 144]) 
encoder.layer1.1: torch.Size([1, 64, 144, 144]) 
encoder.layer1: torch.Size([1, 64, 144, 144]) 
encoder.layer2.0.conv1: 

In [217]:
len(visualisation)

145

In [215]:
visualisation[0]

('encoder.conv1',
 tensor([[[[-1.9353e+00, -5.8528e+00, -2.7648e+00,  ..., -4.1500e+00,
            -9.4235e-02,  1.4513e+00],
           [-3.4575e+00, -3.1630e+00, -8.7083e-01,  ...,  3.1493e+00,
             1.8155e+00,  1.5542e-01],
           [ 2.4868e+00, -2.7244e+00,  3.3991e-01,  ...,  1.3450e+00,
            -2.0022e+00,  5.1955e+00],
           ...,
           [-3.8173e+00,  5.5885e-02, -3.0199e+00,  ..., -9.1131e-01,
            -2.1043e+00, -2.4944e+00],
           [ 5.3915e+00,  3.3375e-02, -7.1988e+00,  ..., -3.0478e+00,
            -2.3478e+00, -4.4943e+00],
           [-1.3364e+00,  1.6976e+00,  5.3142e+00,  ...,  2.7189e+00,
             2.6444e-02,  6.8614e+00]],
 
          [[-3.1401e-01, -6.9064e-01, -2.4634e-01,  ..., -2.2497e+00,
            -9.8976e-01, -2.6008e-01],
           [ 2.2689e+00,  1.6058e+00, -8.8914e-01,  ...,  8.1021e-01,
            -8.8663e-01, -1.6816e+00],
           [-7.7532e-01, -1.4787e+00, -2.2148e+00,  ..., -3.5833e+00,
            -2.7980e+

In [ ]:
VerboseExecution()

In [267]:
for ii, (name, mn,output) in enumerate(visualisation):
    try:
        if len(list(mn.children()))>0:
            print(f'___{name}___')
        else:
            print(f"[{ii}]:{name}<=>{mn.__name__}::{output.size()}:::{mn}")
    except Exception as e:
        print(f"[{ii}]:EXCEPTION {name}:{e}")
#model_resnet18.encoder.conv1


[0]:EXCEPTION encoder.conv1:'Conv2d' object has no attribute '__name__'
[1]:EXCEPTION encoder.bn1:'BatchNorm2d' object has no attribute '__name__'
[2]:EXCEPTION encoder.relu:'ReLU' object has no attribute '__name__'
[3]:EXCEPTION encoder.maxpool:'MaxPool2d' object has no attribute '__name__'
[4]:EXCEPTION encoder.layer1.0.conv1:'Conv2d' object has no attribute '__name__'
[5]:EXCEPTION encoder.layer1.0.bn1:'BatchNorm2d' object has no attribute '__name__'
[6]:EXCEPTION encoder.layer1.0.relu:'ReLU' object has no attribute '__name__'
[7]:EXCEPTION encoder.layer1.0.conv2:'Conv2d' object has no attribute '__name__'
[8]:EXCEPTION encoder.layer1.0.bn2:'BatchNorm2d' object has no attribute '__name__'
[9]:EXCEPTION encoder.layer1.0.relu:'ReLU' object has no attribute '__name__'
___encoder.layer1.0___
[11]:EXCEPTION encoder.layer1.1.conv1:'Conv2d' object has no attribute '__name__'
[12]:EXCEPTION encoder.layer1.1.bn1:'BatchNorm2d' object has no attribute '__name__'
[13]:EXCEPTION encoder.layer1.1

In [235]:
dir(visualisation[110][1])
print(visualisation[110][1].modules)
print(type(visualisation[110][1].modules))

<bound method Module.modules of Conv2dReLU(
  (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU(inplace=True)
)>
<class 'method'>


In [266]:
type(visualisation[127][1])

torch.nn.modules.conv.Conv2d

In [ ]:
print(list(visualisation[109][1].children()))
print(list(visualisation[110][1].children()))
print(list(visualisation[110][1].children()))
print(list(visualisation[109][1].parent()))

In [ ]:
mod_names, mod_modules = [], []
for mix, (name, module) in enumerate(model_resnet18.named_modules()):
    mod_names.append(name)
    mod_modules.append(module)


In [252]:
print(list(visualisation[2][1].children()))

[]


In [144]:
for ii in range(10):
    print((mod_names[ii], type(mod_modules[ii]), isinstance(mod_modules[ii], nn.Sequential)))

('', <class 'segmentation_models_pytorch.decoders.unet.model.Unet'>, False)
('encoder', <class 'segmentation_models_pytorch.encoders.resnet.ResNetEncoder'>, False)
('encoder.conv1', <class 'torch.nn.modules.conv.Conv2d'>, False)
('encoder.bn1', <class 'torch.nn.modules.batchnorm.BatchNorm2d'>, False)
('encoder.relu', <class 'torch.nn.modules.activation.ReLU'>, False)
('encoder.maxpool', <class 'torch.nn.modules.pooling.MaxPool2d'>, False)
('encoder.layer1', <class 'torch.nn.modules.container.Sequential'>, True)
('encoder.layer1.0', <class 'torchvision.models.resnet.BasicBlock'>, False)
('encoder.layer1.0.conv1', <class 'torch.nn.modules.conv.Conv2d'>, False)
('encoder.layer1.0.bn1', <class 'torch.nn.modules.batchnorm.BatchNorm2d'>, False)


In [156]:
for ii in [0, 1, 2, 6, 7, 9]:
    print(type(mod_modules[ii]), [att for att in dir(mod_modules[ii]) if "_" not in att])


<class 'segmentation_models_pytorch.decoders.unet.model.Unet'> ['apply', 'bfloat16', 'buffers', 'children', 'cpu', 'cuda', 'decoder', 'double', 'encoder', 'eval', 'float', 'forward', 'half', 'initialize', 'modules', 'name', 'parameters', 'predict', 'to', 'train', 'training', 'type']
<class 'segmentation_models_pytorch.encoders.resnet.ResNetEncoder'> ['apply', 'bfloat16', 'bn1', 'buffers', 'children', 'conv1', 'cpu', 'cuda', 'dilation', 'double', 'eval', 'float', 'forward', 'groups', 'half', 'inplanes', 'layer1', 'layer2', 'layer3', 'layer4', 'maxpool', 'modules', 'parameters', 'relu', 'to', 'train', 'training', 'type']
<class 'torch.nn.modules.conv.Conv2d'> ['apply', 'bfloat16', 'bias', 'buffers', 'children', 'cpu', 'cuda', 'dilation', 'double', 'eval', 'float', 'forward', 'groups', 'half', 'modules', 'padding', 'parameters', 'stride', 'to', 'train', 'training', 'transposed', 'type', 'weight']
<class 'torch.nn.modules.container.Sequential'> ['apply', 'bfloat16', 'buffers', 'children', 

In [129]:
par_names, par_parameters = [], []
for mix, (name, parameters) in enumerate(model_resnet18.named_parameters()):
    par_names.append(name)
    par_parameters.append(parameters)


In [145]:
for ii in range(10):
    print((par_names[ii], type(par_parameters[ii])))

('encoder.conv1.weight', <class 'torch.nn.parameter.Parameter'>)
('encoder.bn1.weight', <class 'torch.nn.parameter.Parameter'>)
('encoder.bn1.bias', <class 'torch.nn.parameter.Parameter'>)
('encoder.layer1.0.conv1.weight', <class 'torch.nn.parameter.Parameter'>)
('encoder.layer1.0.bn1.weight', <class 'torch.nn.parameter.Parameter'>)
('encoder.layer1.0.bn1.bias', <class 'torch.nn.parameter.Parameter'>)
('encoder.layer1.0.conv2.weight', <class 'torch.nn.parameter.Parameter'>)
('encoder.layer1.0.bn2.weight', <class 'torch.nn.parameter.Parameter'>)
('encoder.layer1.0.bn2.bias', <class 'torch.nn.parameter.Parameter'>)
('encoder.layer1.1.conv1.weight', <class 'torch.nn.parameter.Parameter'>)


In [150]:
print([att for att in dir(par_parameters[8]) if "_" not in att])

['T', 'abs', 'acos', 'add', 'addbmm', 'addcdiv', 'addcmul', 'addmm', 'addmv', 'addr', 'all', 'allclose', 'angle', 'any', 'argmax', 'argmin', 'argsort', 'asin', 'atan', 'atan2', 'backward', 'baddbmm', 'bernoulli', 'bfloat16', 'bincount', 'bmm', 'bool', 'byte', 'ceil', 'char', 'cholesky', 'chunk', 'clamp', 'clone', 'coalesce', 'conj', 'contiguous', 'cos', 'cosh', 'cpu', 'cross', 'cuda', 'cummax', 'cummin', 'cumprod', 'cumsum', 'data', 'dequantize', 'det', 'detach', 'device', 'diag', 'diagflat', 'diagonal', 'digamma', 'dim', 'dist', 'div', 'dot', 'double', 'dtype', 'eig', 'eq', 'equal', 'erf', 'erfc', 'erfinv', 'exp', 'expand', 'expm1', 'fft', 'flatten', 'flip', 'float', 'floor', 'fmod', 'frac', 'gather', 'ge', 'geqrf', 'ger', 'grad', 'gt', 'half', 'hardshrink', 'histc', 'ifft', 'indices', 'int', 'inverse', 'irfft', 'isclose', 'item', 'kthvalue', 'layout', 'le', 'lerp', 'lgamma', 'log', 'log10', 'log1p', 'log2', 'logdet', 'logsumexp', 'long', 'lstsq', 'lt', 'lu', 'matmul', 'max', 'mean', 

In [132]:
print(len(mod_names),len(par_names))

140 92


In [136]:
print(par_names[0:6])
print(mod_names[0:10])


['encoder.conv1.weight', 'encoder.bn1.weight', 'encoder.bn1.bias', 'encoder.layer1.0.conv1.weight', 'encoder.layer1.0.bn1.weight', 'encoder.layer1.0.bn1.bias']
['', 'encoder', 'encoder.conv1', 'encoder.bn1', 'encoder.relu', 'encoder.maxpool', 'encoder.layer1', 'encoder.layer1.0', 'encoder.layer1.0.conv1', 'encoder.layer1.0.bn1']


In [133]:
418-183

235

In [72]:
model_resnet18.encoder.maxpool(model_resnet18.encoder.conv1(torch.randn(1, 1, 576, 576))).shape

torch.Size([1, 64, 144, 144])

In [67]:
model_resnet18.encoder.conv1

Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)

In [44]:
from torch.autograd import Variable
dummy_input = Variable(torch.randn(1, 1, 576, 576))

In [43]:
torch.onnx.export(model_resnet18, dummy_input, os.path.join(model_dir, "unet_resnet18_not_trained.onnx"))

In [45]:
print("___")
print(modules)

___
[Unet(
  (encoder): ResNetEncoder(
    (conv1): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, 

In [40]:
torch.save(modules[1], os.path.join(model_dir, "unet_resnet18_encoder.pth"))

In [ ]:
torch.save(model_resnet18, os.path.join(model_dir, "unet_resnet18_non_trained.pth"))

In [15]:
batch_size = 10
channels= 3
#summary(model_resnet18, input_size=(batch_size,channels, 572, 572))
summary(model_resnet18)

Layer (type:depth-idx)                        Param #
Unet                                          --
├─ResNetEncoder: 1-1                          --
│    └─Conv2d: 2-1                            3,136
│    └─BatchNorm2d: 2-2                       128
│    └─ReLU: 2-3                              --
│    └─MaxPool2d: 2-4                         --
│    └─Sequential: 2-5                        --
│    │    └─BasicBlock: 3-1                   73,984
│    │    └─BasicBlock: 3-2                   73,984
│    └─Sequential: 2-6                        --
│    │    └─BasicBlock: 3-3                   230,144
│    │    └─BasicBlock: 3-4                   295,424
│    └─Sequential: 2-7                        --
│    │    └─BasicBlock: 3-5                   919,040
│    │    └─BasicBlock: 3-6                   1,180,672
│    └─Sequential: 2-8                        --
│    │    └─BasicBlock: 3-7                   3,673,088
│    │    └─BasicBlock: 3-8                   4,720,640
├─UnetDecoder: 1

In [8]:
torch.save(model_resnet18, os.path.join(model_dir, "unet_resnet18_non_trained.pth"))


In [4]:
print(model_resnet18)

Unet(
  (encoder): ResNetEncoder(
    (conv1): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track

In [11]:
model_vgg = smp.Unet(
    encoder_name="vgg16",  # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
    encoder_weights="imagenet",  # use `imagenet` pre-trained weights for encoder initialization
    in_channels=1,  # model input channels (1 for gray-scale images, 3 for RGB, etc.)
    classes=3,  # model output channels (number of classes in your dataset)
)

torch.save(model_vgg, os.path.join(model_dir, "unet_vgg16_non_trained.pth"))

In [12]:
print(model_vgg)

Unet(
  (encoder): VGGEncoder(
    (features): Sequential(
      (0): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU(inplace=True)
      (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (3): ReLU(inplace=True)
      (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (6): ReLU(inplace=True)
      (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (8): ReLU(inplace=True)
      (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (11): ReLU(inplace=True)
      (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (13): ReLU(inplace=True)
      (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (15): ReLU(inplace=True)
      

In [193]:
import torch.nn as nn

class myNet(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv = nn.Conv2d(3,10,2, stride = 2)
    self.relu = nn.ReLU()
    self.flatten = lambda x: x.view(-1)
    self.fc1 = nn.Linear(160,5)
  def forward(self, x):
    x = self.relu(self.conv(x))
    return self.fc1(self.flatten(x))

net = myNet()

def hook_forward_fn(m, i, o):
    #print(x)
    print(dir(m))
    print(m.state_dict)

    #print(m.__annotations__)

    print('_' if not hasattr(m, '_get_name') else m._get_name(), type(m), i[0].size(), o.size())

for m in net.named_modules():
    m[1].register_forward_hook(hook_forward_fn)
#net.conv.register_forward_hook(hook_forward_fn)

inp = torch.randn(1,3,8,8)
out = net(inp)

(1 - out.mean()).backward()

['__annotations__', '__call__', '__class__', '__constants__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattr__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__slotnames__', '__str__', '__subclasshook__', '__weakref__', '_apply', '_backward_hooks', '_buffers', '_conv_forward', '_forward_hooks', '_forward_pre_hooks', '_get_name', '_load_from_state_dict', '_load_state_dict_pre_hooks', '_modules', '_named_members', '_parameters', '_register_load_state_dict_pre_hook', '_register_state_dict_hook', '_replicate_for_data_parallel', '_reversed_padding_repeated_twice', '_save_to_state_dict', '_slow_forward', '_state_dict_hooks', '_version', 'add_module', 'apply', 'bfloat16', 'bias', 'buffers', 'children', 'cpu', 'cuda', 'dilation', 'double', 'dump_patches', 'eval', 'extra_repr',